In [4]:
import pandas as pd
def list_chapter_titles(lang):
    dfjson = pd.read_json(f'~/pi/stack/crawlers/langcrs/all_{lang}.json')
    return [name for name, group in dfjson.groupby('chapter')]
list_chapter_titles('es')

['Activities',
 'Adjectives 1',
 'Adjectives 2',
 'Adjectives 3',
 'Adverbs',
 'Appointment',
 'Around the house',
 'Asking for directions',
 'Asking questions 1',
 'Asking questions 2',
 'At school',
 'At the airport',
 'At the bank',
 'At the cinema',
 'At the doctor',
 'At the post office',
 'At the restaurant 1',
 'At the restaurant 2',
 'At the restaurant 3',
 'At the restaurant 4',
 'At the train station',
 'At the zoo',
 'Beverages',
 'Car breakdown',
 'City tour',
 'Colors',
 'Conjunctions 1',
 'Conjunctions 2',
 'Conjunctions 3',
 'Conjunctions 4',
 'Countries and Languages',
 'Days of the week',
 'Double connectors',
 'En route',
 'Family Members',
 'Feelings',
 'Fruits and food',
 'Genitive',
 'Getting to know others',
 'Going out in the evening',
 'House cleaning',
 'Imperative 1',
 'Imperative 2',
 'In nature',
 'In the city',
 'In the department store',
 'In the discotheque',
 'In the hotel – Arrival',
 'In the hotel – Complaints',
 'In the kitchen',
 'In the swimming poo

In [5]:
lang='es'
dfjson = pd.read_json(f'~/pi/stack/crawlers/langcrs/all_{lang}.json')
dfjson.head()

,chapter,index,text,translate,translit,audio
0,People,1,I,yo,,https://www.book2.nl/book2/ES/SOUND/0003.mp3
1,People,2,I and you,yo y tú,,https://www.book2.nl/book2/ES/SOUND/0004.mp3
2,People,3,both of us,nosotros / nosotras dos,,https://www.book2.nl/book2/ES/SOUND/0005.mp3
3,People,4,he,él,,https://www.book2.nl/book2/ES/SOUND/0006.mp3
4,People,5,he and she,él y ella,,https://www.book2.nl/book2/ES/SOUND/0007.mp3


In [13]:
from sagas.nlu.nlu_tools import vis_tree
from sagas.nlu.ruleset_procs import cached_chunks
chunks = cached_chunks('Nosotros estamos en la escuela.',
                       source='es',
                       engine='stanza')
ds=chunks['root_domains'][0]
vis_tree(ds, 'es', trans=True)

root: estamos(estar; we're, verb, 2)
├── obl: escuela(escuela; school, noun, 5)
│   ├── det: la(el; the, det, 4)
│   └── case: en(en; in, adp, 3)
├── nsubj: Nosotros(yo; We, pron, 1)
└── punct: .(_, punct, 6)


In [33]:
any([])

False

In [52]:
from sagas.nlu.utils import fix_sents
from sagas.nlu.nlu_tools import treeing
from sagas.nlu.features import get_feats_map, feats_map
from anytree import Node, RenderTree, AsciiStyle, Walker, Resolver
from anytree.importer import DictImporter
from anytree.search import findall, findall_by_attr

importer = DictImporter()

def proc_verb_subs(node):
    toks=findall_by_attr(node, name='dependency_relation', value='nsubj')
    if toks:
        tok=toks[0]
        objs=findall(node, filter_=lambda n: n.dependency_relation in ("obj"))
        if objs:
            print('✔[verb-nsubj-obj]', node.text, tok.text, objs[0].text)
            return True
        objs=findall_by_attr(node, name='upos', value='NOUN')
        # objs=findall(f, filter_=lambda node: node.upos in ("NOUN", "ADJ"))
        if objs:
            print('✔[verb-nsubj-noun]', node.text, tok.text, 
                  objs[0].dependency_relation, objs[0].text)  
            return True
    return False
    
def digest(sents, lang, engine='stanza'):
    chunks = cached_chunks(sents,
                       source=lang,
                       engine=engine)
    root=chunks['root_domains']
    ds=treeing(root[0])
    f = importer.import_(ds)
    words=findall_by_attr(f, name='upos', value='VERB')
    if words:
        print(sents, len(words))
        rs=[]
        for w in words:
            rs.append(proc_verb_subs(w))
        print(RenderTree(f, style=AsciiStyle()).by_attr(lambda n: f"{n.dependency_relation}: {n.text} {n.upos}"))
        return any(rs)
    return False

def proc_corpus(lang, chapter):
    dfjson = pd.read_json(f'~/pi/stack/crawlers/langcrs/all_{lang}.json')
    ch=dfjson[dfjson['chapter'].str.match(chapter)]
    rs_map={}
    for i, (sent,ref) in enumerate(zip(ch['translate'], ch['text'])):
        text=fix_sents(sent, lang)
        print('-', i, ref, '-'*10)
        # digest(text, 'spacy')
        rs_map[i]=digest(text, lang, 'stanza')
    return len(ch), len([t for t in rs_map.values() if t])    

proc_corpus('es', 'At school')
# proc_corpus('es', 'Public transportation')
# proc_corpus('fr')
# proc_corpus('de')
# proc_corpus('ru')
# proc_corpus('ja', 'At school')
# proc_corpus('fa', 'At school')
# proc_corpus('id', 'At school')

- 0 Where are we? ----------
Dónde estamos? 1
root: estamos VERB
|-- obl: Dónde PRON
+-- punct: ? PUNCT
- 1 We are at school. ----------
Nosotros estamos en la escuela. 1
✔[verb-nsubj-noun] estamos Nosotros obl escuela
root: estamos VERB
|-- obl: escuela NOUN
|   |-- det: la DET
|   +-- case: en ADP
|-- nsubj: Nosotros PRON
+-- punct: . PUNCT
- 2 We are having class / a lesson. ----------
Nosotros tenemos clase. 1
✔[verb-nsubj-obj] tenemos Nosotros clase
root: tenemos VERB
|-- punct: . PUNCT
|-- nsubj: Nosotros PRON
+-- obj: clase NOUN
- 3 Those are the school children. ----------
- 4 That is the teacher. ----------
- 5 That is the class. ----------
- 6 What are we doing? ----------
Qué hacemos? 1
root: hacemos VERB
|-- punct: ? PUNCT
+-- obj: Qué PRON
- 7 We are learning. ----------
Nosotros estudiamos. 1
root: estudiamos VERB
|-- punct: . PUNCT
+-- nsubj: Nosotros PRON
- 8 We are learning a language. ----------
Nosotros estudiamos un idioma. 1
✔[verb-nsubj-obj] estudiamos Nosotros id

(18, 10)